In [2]:
import pandas as pd
import re
import numpy as np
from IPython.display import display

In [ ]:
df = pd.read_csv(r"D:\GitHubRepos\is6941-ml-social-media\analysis\data\BV1dZwLeKEzG_comments.csv")
df.head()

,用户名,性别,IP地址,评论内容
0,柳橙迷索拉,女,江苏,2017年，NS定价：\n29980日元\n折合当时人民币：2000人民币（注：折合现在的人...
1,阿粥今天吃什么,保密,湖南,省流：\n外观与爆料一致\n名字为ns2\n演示游戏为马车\n确认ns卡带可以在ns2上使用...
2,Yuri-Sense,保密,湖北,ns2的掌机模式，\n性能是前代掌机模式的4.35倍，\n是前代主机模式的3.4倍。\n\n...
3,宛如繁星_,男,广东,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...
4,Go-liath,男,天津,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...


In [234]:
# 查看并处理缺失值
df.isnull().sum()

用户名     0
性别      0
IP地址    0
评论内容    0
dtype: int64

In [235]:
# 去除重复数据
duplicate_mask = df.duplicated(subset=['用户名', '评论内容'], keep='first')
print(f"\n发现重复数据：{duplicate_mask.sum()}条")


发现重复数据：2031条


In [236]:
df = df[~duplicate_mask]

In [237]:
# 标准化性别字段
gender_mapping = {
    '女': 'female',
    '男': 'male',
    '保密': 'unknown',
    'unknown': 'unknown'
}
df['性别'] = df['性别'].map(gender_mapping).fillna('unknown')

In [238]:
df['性别'].value_counts()

性别
unknown    3289
male       2656
female      388
Name: count, dtype: int64

In [239]:
# 清理评论特殊符号
def clean_comment(text):
    # 去除B站表情符号 [笑哭] [doge_金箍] 等
    text = re.sub(r'$$.\*?$$', '', text)
    # 去除HTML符号
    text = re.sub(r'&[a-z]+;', '', text)
    # 去除多余换行符和空格
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # 去除首尾空白
    return text.strip()

In [240]:
df['清洗后评论'] = df['评论内容'].apply(clean_comment)

In [241]:
display(df.head())

,用户名,性别,IP地址,评论内容,清洗后评论
0,柳橙迷索拉,female,江苏,2017年，NS定价：\n29980日元\n折合当时人民币：2000人民币（注：折合现在的人...,2017年，NS定价： 29980日元 折合当时人民币：2000人民币（注：折合现在的人民币...
1,阿粥今天吃什么,unknown,湖南,省流：\n外观与爆料一致\n名字为ns2\n演示游戏为马车\n确认ns卡带可以在ns2上使用...,省流： 外观与爆料一致 名字为ns2 演示游戏为马车 确认ns卡带可以在ns2上使用 4月2...
2,Yuri-Sense,unknown,湖北,ns2的掌机模式，\n性能是前代掌机模式的4.35倍，\n是前代主机模式的3.4倍。\n\n...,ns2的掌机模式， 性能是前代掌机模式的4.35倍， 是前代主机模式的3.4倍。 ns2的主...
3,宛如繁星_,male,广东,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...
4,Go-liath,male,天津,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...


In [242]:
# IP地址标准化
def standardize_ip(ip):
    # 识别国内省份
    provinces = ['北京','天津','上海','重庆','河北','山西','辽宁','吉林','黑龙江','江苏','浙江','安徽','福建',
                 '江西','山东','河南','湖北','湖南','广东','海南','四川','贵州','云南','陕西','甘肃','青海',
                 '中国台湾','内蒙古','广西','西藏','宁夏','新疆','中国香港','中国澳门']
    
    if ip in provinces:
        return f'中国-{ip}'
    elif ip == '未知':
        return '未知'
    else:
        return f'海外-{ip}'

In [243]:
df['标准化IP'] = df['IP地址'].apply(standardize_ip)

In [244]:
display(df.head(10))

,用户名,性别,IP地址,评论内容,清洗后评论,标准化IP
0,柳橙迷索拉,female,江苏,2017年，NS定价：\n29980日元\n折合当时人民币：2000人民币（注：折合现在的人...,2017年，NS定价： 29980日元 折合当时人民币：2000人民币（注：折合现在的人民币...,中国-江苏
1,阿粥今天吃什么,unknown,湖南,省流：\n外观与爆料一致\n名字为ns2\n演示游戏为马车\n确认ns卡带可以在ns2上使用...,省流： 外观与爆料一致 名字为ns2 演示游戏为马车 确认ns卡带可以在ns2上使用 4月2...,中国-湖南
2,Yuri-Sense,unknown,湖北,ns2的掌机模式，\n性能是前代掌机模式的4.35倍，\n是前代主机模式的3.4倍。\n\n...,ns2的掌机模式， 性能是前代掌机模式的4.35倍， 是前代主机模式的3.4倍。 ns2的主...,中国-湖北
3,宛如繁星_,male,广东,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...,中国-广东
4,Go-liath,male,天津,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...,中国-天津
5,Refha,male,俄罗斯,只能说一代的滑插设计无论是从标志上来说还是实际上应用上来说都太有代表性了 导致影响大到老任第...,只能说一代的滑插设计无论是从标志上来说还是实际上应用上来说都太有代表性了 导致影响大到老任第...,海外-俄罗斯
6,房石诶哆,male,江苏,怎么说呢，作为任天堂粉丝我很激动，但我最推荐的还是不买首发，先继续用switch玩（因为我个...,怎么说呢，作为任天堂粉丝我很激动，但我最推荐的还是不买首发，先继续用switch玩（因为我个...,中国-江苏
7,可达鸭抢饭团,male,北京,我喜欢lite的小巧和轻便，还希望能连接在电视上大屏幕玩，希望出个小屏能连电视的，为什么就不...,我喜欢lite的小巧和轻便，还希望能连接在电视上大屏幕玩，希望出个小屏能连电视的，为什么就不...,中国-北京
8,Fimmm520,unknown,浙江,3T多浮点性能+12GB内存+DLSS，第三方厂商没在任天堂主机上打过这么富裕的仗[笑哭],3T多浮点性能+12GB内存+DLSS，第三方厂商没在任天堂主机上打过这么富裕的仗[笑哭],中国-浙江
9,猫神教小鱼干总管吖,male,江西,看快拆的设计方式，承重应该是和被子角固定器那样的强磁铁吸附，靠纵向力按开，横向力非常大，应该...,看快拆的设计方式，承重应该是和被子角固定器那样的强磁铁吸附，靠纵向力按开，横向力非常大，应该...,中国-江西


In [245]:
df.describe()

,用户名,性别,IP地址,评论内容,清洗后评论,标准化IP
count,6333,6333,6333,6333,6333,6333
unique,5935,3,67,6039,6039,67
top,账号已注销,unknown,广东,见证历史,见证历史,中国-广东
freq,15,3289,892,54,54,892


In [246]:
# 处理异常评论
# 定义异常评论规则
def is_valid_comment(text):
    # 有效评论长度阈值
    if len(text) < 3:
        return False
    # 包含过多数字
    if len(re.findall(r'\d', text)) / len(text) > 0.3:
        return False
    return True

In [247]:
# 添加有效性标记
df['是否有效'] = df['清洗后评论'].apply(is_valid_comment)
print(f"\n无效评论数量：{len(df[~df['是否有效']])}")


无效评论数量：318


In [248]:
# 保存清洗结果
clean_df = df[df['是否有效']].drop(columns=['是否有效', 'IP地址', '评论内容'])
clean_df = clean_df.rename(columns={
    '清洗后评论': '评论内容',
    '标准化IP': 'IP属地'
})

In [249]:
display(clean_df.head(10))

,用户名,性别,评论内容,IP属地
0,柳橙迷索拉,female,2017年，NS定价： 29980日元 折合当时人民币：2000人民币（注：折合现在的人民币...,中国-江苏
1,阿粥今天吃什么,unknown,省流： 外观与爆料一致 名字为ns2 演示游戏为马车 确认ns卡带可以在ns2上使用 4月2...,中国-湖南
2,Yuri-Sense,unknown,ns2的掌机模式， 性能是前代掌机模式的4.35倍， 是前代主机模式的3.4倍。 ns2的主...,中国-湖北
3,宛如繁星_,male,必买首发，想体验下法环在ns2运行是什么感觉，还有今年的宝可梦za。感觉今年也是个游戏大年啊...,中国-广东
4,Go-liath,male,有个细节 变成鼠标功能的时候下面是加了底座的，然后后面有又分离了，我是说突出来一块直接接触桌...,中国-天津
5,Refha,male,只能说一代的滑插设计无论是从标志上来说还是实际上应用上来说都太有代表性了 导致影响大到老任第...,海外-俄罗斯
6,房石诶哆,male,怎么说呢，作为任天堂粉丝我很激动，但我最推荐的还是不买首发，先继续用switch玩（因为我个...,中国-江苏
7,可达鸭抢饭团,male,我喜欢lite的小巧和轻便，还希望能连接在电视上大屏幕玩，希望出个小屏能连电视的，为什么就不...,中国-北京
8,Fimmm520,unknown,3T多浮点性能+12GB内存+DLSS，第三方厂商没在任天堂主机上打过这么富裕的仗[笑哭],中国-浙江
9,猫神教小鱼干总管吖,male,看快拆的设计方式，承重应该是和被子角固定器那样的强磁铁吸附，靠纵向力按开，横向力非常大，应该...,中国-江西


In [250]:
clean_df.describe()

,用户名,性别,评论内容,IP属地
count,6015,6015,6015,6015
unique,5657,3,5804,65
top,账号已注销,unknown,见证历史,中国-广东
freq,15,3109,54,853


In [251]:
clean_df.to_csv('D:\GitHubRepos\is6941-ml-social-media\\analysis\data\cleaned_BV1dZwLeKEzG_comments.csv', index=False, encoding='utf_8_sig')

<>:1: SyntaxWarning: invalid escape sequence '\G'
<>:1: SyntaxWarning: invalid escape sequence '\G'
C:\Users\wjw20\AppData\Local\Temp\ipykernel_7352\2832142298.py:1: SyntaxWarning: invalid escape sequence '\G'
  clean_df.to_csv('D:\GitHubRepos\is6941-ml-social-media\\analysis\data\cleaned_BV1dZwLeKEzG_comments.csv', index=False, encoding='utf_8_sig')
